In [27]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
from tqdm import tqdm

In [28]:
import matplotlib.pyplot as plt
import requests

from ratelimit import limits, sleep_and_retry


class SecAPI(object):
    SEC_CALL_LIMIT = {'calls': 10, 'seconds': 1}

    @staticmethod
    @sleep_and_retry
    # Dividing the call limit by half to avoid coming close to the limit
    @limits(calls=SEC_CALL_LIMIT['calls'] / 2, period=SEC_CALL_LIMIT['seconds'])
    def _call_sec(url):
        return requests.get(url)

    def get(self, url):
        return self._call_sec(url).text


def print_ten_k_data(ten_k_data, fields, field_length_limit=50):
    indentation = ' '

    print('[')
    for ten_k in ten_k_data:
        print_statement = '{}{{'.format(indentation)
        for field in fields:
            value = str(ten_k[field])

            # Show return lines in output
            if isinstance(value, str):
                value_str = '\'{}\''.format(value.replace('\n', '\\n'))
            else:
                value_str = str(value)

            # Cut off the string if it gets too long
            if len(value_str) > field_length_limit:
                value_str = value_str[:field_length_limit] + '...'

            print_statement += '\n{}{}: {}'.format(indentation * 2, field, value_str)

        print_statement += '},'
        print(print_statement)
    print(']')


def plot_similarities(similarities_list, dates, title, labels):
    assert len(similarities_list) == len(labels)

    plt.figure(1, figsize=(10, 7))
    for similarities, label in zip(similarities_list, labels):
        plt.title(title)
        plt.plot(dates, similarities, label=label)
        plt.legend()
        plt.xticks(rotation=90)

    plt.show()

In [29]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/yyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/yyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Download 10k / 40F

In [30]:
import pyodbc
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

conCloud = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};server=10.60.23.7;database=CIC;uid=xxx;pwd=xxx")

sql = '''SELECT [ticker]
      ,[yahooTicker]
      ,[industry]
  FROM [CIC].[dbo].[security]
  WHERE industry like 'HC%'
        '''
tickers = pd.read_sql_query(sql, conCloud, parse_dates=True)
HC_tickers = list(tickers[tickers['industry'].str.startswith('HC')]['ticker'])
#HC_tickers=['BAX']

In [4]:
# CIK code lookup
import re
from requests import get

URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
CIK_RE = re.compile(r'.*CIK=(\d{10}).*')

cik_lookup = {}

for ticker in HC_tickers:
    f = requests.get(URL.format(ticker), stream = True)
    results = CIK_RE.findall(f.text)
    if len(results):
        cik_lookup[str(ticker).lower()] = str(results[0])

In [ ]:
sec_api = SecAPI()
from bs4 import BeautifulSoup
def get_sec_data(cik, doc_type, start=0, count=60):
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
        .format(cik, doc_type, start, count)
    sec_data = sec_api.get(rss_url)
    feed = BeautifulSoup(sec_data.encode('ascii'), 'xml').feed
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)]
    return entries

In [ ]:
import re
def get_documents(text):
    extracted_docs = []
    
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    
    doc_start_is = [x.end() for x in      doc_start_pattern.finditer(text)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(text)]
    
    for doc_start_i, doc_end_i in zip(doc_start_is, doc_end_is):
            extracted_docs.append(text[doc_start_i:doc_end_i])
    
    return extracted_docs

In [ ]:
def get_document_type(doc):
    type_pattern = re.compile(r'<TYPE>[^\n]+')    
    doc_type = type_pattern.findall(doc)[0][len('<TYPE>'):]     
    return doc_type.lower()

# Text preprocessing

In [ ]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    return text

def remove_tagcontents(soup,tagname):
    for tag in soup.findAll(tagname):
    
        contents = tag.contents
        parent = tag.parent
        tag.extract()
        for tag in contents:
            parent.append(tag)    
    return soup

def remove_ascii(soup):
    tags = ['GRAPHIC','ZIP','EXCEL','JSON','PDF']
    for tagname in tags:
        for tag in soup.findAll(tagname):
            tag.extract()
    return soup

def replace(text):
    text = text.replace("&nbsp;", " ")
    text = text.replace("&#160;", " ")
    text = text.replace("\xa0", " ")
    text = text.replace("\n", " ")
    text = text.replace("&AMP;", "&")
    text = text.replace("&#38", "&")
    return text

def tag_exhibit(text):
    text = text.replace("\n exhibit number\n", "\n <exhibit>")
def clean_text(text):
    text = text.lower()
    soup = BeautifulSoup(text, 'lxml')
    tags = ['TABLE','TR','TD','DIV','FONT','XML','XBRL','SEC-HEADER','IMS-HEADER']
    for tagname in tags:
        soup = remove_tagcontents(soup,tagname)   
    soup = remove_ascii(soup)
    text = soup.get_text()
    text = replace(text)
    return text
'''
def clean_text(text):
    text = text.lower()
    text = remove_html_tags(text)
    
    return text
'''

# Sentiment analysis

In [ ]:
from sklearn.metrics import jaccard_similarity_score
def get_jaccard_similarity(bag_of_words_matrix):
    
    jaccard_similarities = []
    bag_of_words_matrix = np.array(bag_of_words_matrix, dtype=bool)
    
    for i in range(len(bag_of_words_matrix)-1):
        u = bag_of_words_matrix[i]
        v = bag_of_words_matrix[i+1]
              
        jaccard_similarities.append(jaccard_similarity_score(u,v))
    
    return jaccard_similarities


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def lemmatize_words(words):

    lemmatized_words = [WordNetLemmatizer().lemmatize(word, 'v') for word in words]

    return lemmatized_words

In [ ]:
sentiments = ['negative', 'positive', 'uncertainty', 'litigious', 'constraining', 'interesting']

sentiment_df = pd.read_csv('LoughranMcDonald_MasterDictionary_2018.csv')
sentiment_df.columns = [column.lower() for column in sentiment_df.columns] # Lowercase the columns for ease of use

# Remove unused information
sentiment_df = sentiment_df[sentiments + ['word']]
sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
sentiment_df = sentiment_df[(sentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())
sentiment_df = sentiment_df.drop_duplicates('word')

sentiment_df.head()

In [ ]:
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
def get_bag_of_words(sentiment_words, docs):

    vec = CountVectorizer(vocabulary=sentiment_words)
    vectors = vec.fit_transform(docs)
    words_list = vec.get_feature_names()
    bag_of_words = np.zeros([len(docs), len(words_list)])
    
    for i in range(len(docs)):
        bag_of_words[i] = vectors[i].toarray()[0]
    return bag_of_words.astype(int)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def get_tfidf(sentiment_words, docs):
    
    vec = TfidfVectorizer(vocabulary=sentiment_words)
    tfidf = vec.fit_transform(docs)
    
    return tfidf.toarray()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(tfidf_matrix):
    
    cosine_similarities = []
    
    for i in range(len(tfidf_matrix)-1):
        
        cosine_similarities.append(cosine_similarity(tfidf_matrix[i].reshape(1, -1),tfidf_matrix[i+1].reshape(1, -1))[0,0])
    
    return cosine_similarities


## Main

In [ ]:
import urllib
import sqlalchemy
existed = list(pd.read_sql_query("select distinct ticker from Sentiment.dbo.sentiment_positive", conCloud, parse_dates=True)['ticker'])
for ticker, cik in cik_lookup.items():
    if ticker not in existed:
        sec_data = {}
        sec_data[ticker] = get_sec_data(cik, '10-K')
        if not sec_data[ticker]:
            sec_data[ticker] = get_sec_data(cik, '40-F')
        #pprint.pprint(sec_data[example_ticker][:5])
        example_ticker=ticker

        raw_fillings_by_ticker = {}
        for ticker, data in sec_data.items():
            raw_fillings_by_ticker[ticker] = {}
            for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
                if (file_type in  ('10-K','40-F')):
                    file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')

                    raw_fillings_by_ticker[ticker][file_date] = sec_api.get(file_url)
        #print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

        filling_documents_by_ticker = {}
        for ticker, raw_fillings in raw_fillings_by_ticker.items():
            filling_documents_by_ticker[ticker] = {}
            for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
                filling_documents_by_ticker[ticker][file_date] = get_documents(filling)
        '''
        print('\n\n'.join([
            'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
            for file_date, docs in filling_documents_by_ticker[example_ticker].items()
            for doc_i, doc in enumerate(docs)][:3]))
        '''

        #Filter documents which are not 10K
        ten_ks_by_ticker = {}
        for ticker, filling_documents in filling_documents_by_ticker.items():
            ten_ks_by_ticker[ticker] = []
            for file_date, documents in filling_documents.items():
                for document in documents:
                    if get_document_type(document) == '10-k':
                        ten_ks_by_ticker[ticker].append({
                            'cik': cik_lookup[ticker],
                            'file': document,
                            'file_date': file_date})
        #print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])

        from operator import itemgetter
        ten_ks_by_ticker[example_ticker] = sorted(ten_ks_by_ticker[example_ticker], key=itemgetter('file_date')) 

        for ticker, ten_ks in ten_ks_by_ticker.items():
            for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
                ten_k['file_clean'] = clean_text(ten_k['file'])
        #print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean'])


        word_pattern = re.compile('\w+')
        for ticker, ten_ks in ten_ks_by_ticker.items():
            for ten_k in tqdm(ten_ks, desc='Lemmatize {} 10-Ks'.format(ticker), unit='10-K'):
                ten_k['file_lemma'] = lemmatize_words(word_pattern.findall(ten_k['file_clean']))
        #print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_lemma'])

        sentiment_bow_ten_ks = {}
        for ticker, ten_ks in ten_ks_by_ticker.items():
            lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]

            sentiment_bow_ten_ks[ticker] = {
                sentiment: get_bag_of_words(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
                for sentiment in sentiments}
        #print_ten_k_data([sentiment_bow_ten_ks[example_ticker]], sentiments)

        #delete stop words
        for ticker, ten_ks in ten_ks_by_ticker.items():
            for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
                ten_k['file_clean'] = clean_text(ten_k['file'])
        #print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean'])

        # Get dates for the universe
        file_dates = {
            ticker: [ten_k['file_date'] for ten_k in ten_ks]
            for ticker, ten_ks in ten_ks_by_ticker.items()}

        #jaccard_sim
        jaccard_similarities = {
            ticker: {
                sentiment_name: get_jaccard_similarity(sentiment_values)
                for sentiment_name, sentiment_values in ten_k_sentiments.items()}
            for ticker, ten_k_sentiments in sentiment_bow_ten_ks.items()}
        '''
        plot_similarities(
            [jaccard_similarities[example_ticker][sentiment] for sentiment in sentiments],
            file_dates[example_ticker][1:],
            'Jaccard Similarities for {} Sentiment'.format(example_ticker),
            sentiments)
        '''

        #tfidf
        sentiment_tfidf_ten_ks = {}
        for ticker, ten_ks in ten_ks_by_ticker.items():
            lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
            if not lemma_docs:
                continue

            sentiment_tfidf_ten_ks[ticker] = {
                sentiment: get_tfidf(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
                for sentiment in sentiments}
        #print_ten_k_data([sentiment_tfidf_ten_ks[example_ticker]], sentiments)

        #cosine_sim
        if not sentiment_tfidf_ten_ks:
            continue
        cosine_similarities = {
            ticker: {
                sentiment_name: get_cosine_similarity(sentiment_values)
                for sentiment_name, sentiment_values in ten_k_sentiments.items()}
            for ticker, ten_k_sentiments in sentiment_tfidf_ten_ks.items()}
        '''
        plot_similarities(
            [cosine_similarities[example_ticker][sentiment] for sentiment in sentiments],
            file_dates[example_ticker][1:],
            'Cosine Similarities for {} Sentiment'.format(example_ticker),
            sentiments)
        '''

        positive = pd.DataFrame(columns=['DataDate','Ticker','Jaccard_Similarity','Cosine_similarity_TFIDF'])
        negative = pd.DataFrame(columns=['DataDate','Ticker','Jaccard_Similarity','Cosine_similarity_TFIDF'])
        uncertainty = pd.DataFrame(columns=['DataDate','Ticker','Jaccard_Similarity','Cosine_similarity_TFIDF'])
        interesting = pd.DataFrame(columns=['DataDate','Ticker','Jaccard_Similarity','Cosine_similarity_TFIDF'])
        constraining = pd.DataFrame(columns=['DataDate','Ticker','Jaccard_Similarity','Cosine_similarity_TFIDF'])
        litigious = pd.DataFrame(columns=['DataDate','Ticker','Jaccard_Similarity','Cosine_similarity_TFIDF'])

        positive['DataDate']=file_dates[example_ticker][1:]
        positive['Jaccard_Similarity']=jaccard_similarities[example_ticker]['positive']
        positive['Cosine_similarity_TFIDF']=cosine_similarities[example_ticker]['positive']
        positive['Ticker']=[example_ticker]*len(positive)

        params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};server=10.60.23.7;database=Sentiment;uid=yyan;pwd=3WzvVtn7BqC6!")
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
        # write the DataFrame to a table in the sql database
        positive.to_sql("sentiment_positive", engine, if_exists='append')       #, if_exists='replace'

        negative['DataDate']=file_dates[example_ticker][1:]
        negative['Jaccard_Similarity']=jaccard_similarities[example_ticker]['negative']
        negative['Cosine_similarity_TFIDF']=cosine_similarities[example_ticker]['negative']
        negative['Ticker']=[example_ticker]*len(negative)
        negative.to_sql("sentiment_negative", engine, if_exists='append')

        uncertainty['DataDate']=file_dates[example_ticker][1:]
        uncertainty['Jaccard_Similarity']=jaccard_similarities[example_ticker]['uncertainty']
        uncertainty['Cosine_similarity_TFIDF']=cosine_similarities[example_ticker]['uncertainty']
        uncertainty['Ticker']=[example_ticker]*len(negative)
        uncertainty.to_sql("sentiment_uncertainty", engine, if_exists='append')

        interesting['DataDate']=file_dates[example_ticker][1:]
        interesting['Jaccard_Similarity']=jaccard_similarities[example_ticker]['interesting']
        interesting['Cosine_similarity_TFIDF']=cosine_similarities[example_ticker]['interesting']
        interesting['Ticker']=[example_ticker]*len(interesting)
        interesting.to_sql("sentiment_interesting", engine, if_exists='append')

        constraining['DataDate']=file_dates[example_ticker][1:]
        constraining['Jaccard_Similarity']=jaccard_similarities[example_ticker]['constraining']
        constraining['Cosine_similarity_TFIDF']=cosine_similarities[example_ticker]['interesting']
        constraining['Ticker']=[example_ticker]*len(constraining)
        constraining.to_sql("sentiment_constraining", engine, if_exists='append')

        litigious['DataDate']=file_dates[example_ticker][1:]
        litigious['Jaccard_Similarity']=jaccard_similarities[example_ticker]['litigious']
        litigious['Cosine_similarity_TFIDF']=cosine_similarities[example_ticker]['litigious']
        litigious['Ticker']=[example_ticker]*len(litigious)
        litigious.to_sql("sentiment_litigious", engine, if_exists='append')
